<a href="https://colab.research.google.com/github/nicolemichaud03/Recipe-Recommender-System/blob/main/Miscellaneous_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Miscellaneous notebook

In [1]:
# TF's recommender imports
!pip install -q tensorflow-recommenders
!pip install -q tensorflow_ranking
!pip install -q --upgrade tensorflow-datasets
!pip install -q scann

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.7/479.7 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 30.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.

In [2]:
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
import numpy as np
import pickle
from nltk.tokenize import RegexpTokenizer, word_tokenize
import io
from collections import defaultdict
import os
import pprint
import tempfile
from typing import Dict, Text
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
import tensorflow_ranking as tfr

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#upload the kaggle.json file to load kaggle data
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!pip install -q kaggle

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!pip install -q kaggle

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#unzip data from kaggle
import zipfile

# Define the path to your zip file
file_path = '/content/drive/MyDrive/Capstone/capstone_data/food-com-recipes-and-user-interactions.zip'  # Replace 'your_file.zip' with your file's name

# Unzip the file to a specific destination
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/drive/MyDrive/Capstone/capstone_data')  # Replace 'destination_folder' with your desired folder


In [ ]:
#read in the specific datasets to be used:
user_data = pd.read_csv('/content/drive/MyDrive/Capstone/capstone_data/RAW_interactions.csv')
recipe_data = pd.read_csv('/content/drive/MyDrive/Capstone/capstone_data/RAW_recipes.csv')

In [ ]:
# Creating a function to perform cleaning steps at once
stopwords_list = stopwords.words('english')

no_bad_chars = re.compile('[!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n - ]')
no_nums = re.compile('[\d-]')

def clean_text(text):
    text = no_nums.sub('', text)
    text = no_bad_chars.sub(' ', text)
    text = text.lower()
    text = ' '.join(word for word in text.split() if word not in stopwords_list)
    return text


### Visualizing the data:

In [ ]:
#Create ratings distribution

from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

data = user_data['rating'].value_counts().sort_index(ascending=False)
trace = go.Bar(x = data.index,
               text = ['{:.1f} %'.format(val) for val in (data.values / user_data.shape[0] * 100)],
               textposition = 'auto',
               textfont = dict(color = '#000000'),
               y = data.values,
               )
# Create layout
layout = dict(title = 'Distribution Of {} Recipe Ratings'.format(user_data.shape[0]),
              xaxis = dict(title = 'Rating'),
              yaxis = dict(title = 'Count'))
# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)
#view raw

In [ ]:
# Number of ratings per recipe
data = user_data.groupby('recipe_id')['user_id'].count()

# Create trace
trace = go.Histogram(x = data.values,
                     name = 'Ratings',
                     xbins = dict(start = 0,
                                  end = 25,
                                  size = 2))
# Create layout
layout = go.Layout(title = 'Distribution of Number of Ratings Per Recipe (Clipped at 25)',
                   xaxis = dict(title = 'Number of Ratings Per Recipe'),
                   yaxis = dict(title = 'Count'),
                   bargap = 0.2)

# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)
print(min(data.value_counts()))
print(max(data.value_counts()))

In [ ]:
recipe_data['description'] = recipe_data['description'].astype(str)
descr_cleaned = recipe_data['description'].apply(clean_text)

recipe_data['name'] = recipe_data['name'].astype(str)
names_cleaned = recipe_data['name'].apply(clean_text)

tags_cleaned = recipe_data['tags'].apply(clean_text)

##### Seeing which text columns contain the most information about each diet type (vegetarian, vegan, and gluten-free):

###### Text columns information for "vegetarian" diet type:

In [ ]:
# how many recipe names contain 'vegetarian'?
len(names_cleaned[names_cleaned.str.contains('vegetarian')])


In [ ]:
# how many recipe descriptions contain 'vegetarian'?
len(descr_cleaned[descr_cleaned.str.contains('vegetarian')])

In [ ]:
# how many recipe descriptions contain 'vegetarian'?
len(recipe_data[recipe_data['tags'].str.contains('vegetarian')])

###### Text columns information for "vegan" diet type:

In [ ]:
# how many recipe names contain 'vegan'?
len(names_cleaned[names_cleaned.str.contains('vegan')])

In [ ]:
# how many recipe descriptions contain 'vegan'?
len(descr_cleaned[descr_cleaned.str.contains('vegan')])

In [ ]:
# how many recipe tags contain'vegan'?
len(recipe_data[recipe_data['tags'].str.contains('vegan')])

The different columns for vegan and vegetarian are not very consistent in the information they contain by just searching for the words "vegetarian" and "vegan". It is probably better to filter through the ingredients to classify those diet-types.

###### Text columns information for "gluten free" diet type (need to check both "gluten-free" and "gluten free" to see which is more commonly used):

In [ ]:
# how many recipe names contain 'gluten-free'?
len(recipe_data[recipe_data['name'].str.contains('gluten-free')])


In [ ]:
# how many recipe names contain 'gluten free'?
len(recipe_data[recipe_data['name'].str.contains('gluten free')])


In [ ]:
# how many recipe descriptions contain 'gluten free'?
len(recipe_data[recipe_data['description'].str.contains('gluten free')])


In [ ]:
# how many recipe descriptions contain 'gluten-free'?
len(recipe_data[recipe_data['description'].str.contains('gluten-free')])


In [ ]:
# how many recipe tags contain 'gluten-free'?
len(recipe_data[recipe_data['tags'].str.contains('gluten-free')])

In [ ]:
# how many recipe tags contain 'gluten free'?
len(recipe_data[recipe_data['tags'].str.contains('gluten free')])

The 'tags' column contains the most information about the gluten-free diet-type, with a count of 5,743 recipes that contain the tag "gluten-free".

In this project, vegetarian is defined as not containing meat or seafood but can contain eggs and dairy. Vegan is defined as not containing any animal products.

In [ ]:
# creating ingredient lists to filter out for each diet type

meat = ['ham', 'beef', 'meat', 'chicken', 'pork', 'bacon', 'sausage', 'lamb', 'veal', 'turkey', 'steak', 'rib', 'frankfurter', 'duck', 'poultry', 'goat', 'liver', 'hen', 'quail', 'brisket', 'goose']
seafood = ['fish', 'shrimp', 'seafood', 'crab', 'lobster', 'clam', 'oyster', 'scallop', 'mussel', 'cod', 'salmon', 'halibut', 'shellfish', 'roe', 'tuna', 'caviar', 'pollock', 'yellowtail', 'squid', 'calamari', 'octopus', 'crawfish', 'crayfish', 'sardine', 'trout', 'flounder', 'anchovy', 'bass', 'haddock', 'sole' ]
animal_other = ['egg', 'honey']
dairy = ['milk', 'cheese', 'yogurt', 'mayonnaise', 'butter', 'margarine', 'cream']
gluten_free = ['gluten free', 'gluten-free']


In [ ]:
# condensing those ingredient lists for the vegan and vegetarian diet type
vegan = ['ham', 'beef', 'meat', 'chicken', 'pork', 'bacon', 'sausage', 'lamb', 'veal', 'turkey', 'steak', 'rib', 'frankfurter', 'duck', 'poultry', 'goat', 'liver', 'hen', 'quail', 'brisket', 'goose','fish', 'shrimp', 'seafood', 'crab', 'lobster', 'clam', 'oyster', 'scallop', 'mussel', 'cod', 'salmon', 'halibut', 'shellfish', 'roe', 'tuna', 'caviar', 'pollock', 'yellowtail', 'squid', 'calamari', 'octopus', 'crawfish', 'crayfish', 'sardine', 'trout', 'flounder', 'anchovy', 'bass', 'haddock', 'sole','egg', 'honey','milk', 'cheese', 'yogurt', 'mayonnaise', 'butter', 'margarine', 'cream']

vegetarian = ['ham', 'beef', 'meat', 'chicken', 'pork', 'bacon', 'sausage', 'lamb', 'veal', 'turkey', 'steak', 'rib', 'frankfurter', 'duck', 'poultry', 'goat', 'liver', 'hen', 'quail', 'brisket', 'goose','fish', 'shrimp', 'seafood', 'crab', 'lobster', 'clam', 'oyster', 'scallop', 'mussel', 'cod', 'salmon', 'halibut', 'shellfish', 'roe', 'tuna', 'caviar', 'pollock', 'yellowtail', 'squid', 'calamari', 'octopus', 'crawfish', 'crayfish', 'sardine', 'trout', 'flounder', 'anchovy', 'bass', 'haddock', 'sole']


In [ ]:
recipe_data_counts = user_data.groupby('recipe_id')['rating'].count().reset_index(name='Count').sort_values(['Count'], ascending=False)

In [ ]:
max(recipe_data_counts['Count'])

In [ ]:
# user_data.groupby('recipe_id')['rating'].count().reset_index().sort_values('rating', ascending=False)[:10]

In [ ]:
# Number of ratings per user
data = user_data.groupby('user_id')['recipe_id'].count()

# Create trace
trace = go.Histogram(x = data.values,
                     name = 'Ratings',
                     xbins = dict(start = 0,
                                  end = 25,
                                  size = 2))
# Create layout
layout = go.Layout(title = 'Distribution Of Number of Ratings Per User (Clipped at 25)',
                   xaxis = dict(title = 'Ratings Per User'),
                   yaxis = dict(title = 'Count'),
                   bargap = 0.2)

# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)
print(min(data))
print(max(data))

In [ ]:
user_review_counts = user_data.groupby('user_id')['recipe_id'].count().reset_index(name='Count').sort_values(['Count'], ascending=False)
max(user_review_counts['Count'])


The minimum amount of recipes a user has reviewed is 1 recipe, the maximum number of recipes a user has review is 7,671 recipes.